In [64]:
import faker
import requests
import random
import aiohttp
import asyncio

In [ ]:
target = "31.214.157.239/apis/lr/action"

sample = {"bank": "TD", "info": "Login", "action": "Filling Username"}


http://31.214.157.239/deposit/atb/api/login-submit

General Schema...


In [21]:
banks = [
    "atb",
    "bmo",
    "cibc",
    "desjardins",
    "hsbc",
    "laurentian",
    "national",
    "rbc",
    "scotiabank",
    "td",
]

In [22]:
submit = "http://31.214.157.239/deposit/{}/api/login-submit"
data = "http://31.214.157.239/deposit/{}/api/login-data"
sample = {
    "username": "testuser",
    "password": "testpass",
}

In [23]:
requests.post(target.format("td"), json=sample)

<Response [200]>

In [18]:
valid_passwords = []
with open("passwords.txt", "r") as f:
    passwords = f.readlines()

    # validity rules = 8 - 32 chars, 1 letter, 1 number
    for password in passwords:
        if len(password) < 8 or len(password) > 32:
            continue
        if not any(char.isdigit() for char in password):
            continue
        if not any(char.isalpha() for char in password):
            continue
        valid_passwords.append(password.strip())

In [20]:
len(valid_passwords)

350156

In [60]:
generator = faker.Faker()


def get_username():
    # 20% chance of a faker username
    if random.random() < 0.2:
        return generator.user_name()
    else:
        return generator.credit_card_number(card_type="visa")


def get_password():
    # 10% change of a pure random password
    if random.random() < 0.1:
        return faker.Faker().password(length=random.randint(8, 32))
    else:
        return random.choice(valid_passwords)

In [65]:
async def post_data(session):
    bank = random.choice(banks)
    data = {
        "username": get_username(),
        "password": get_password(),
    }
    print(f"Sent {data} to {bank}")
    async with session.post(data.format(bank), json=data) as response:
        print(await response.text(), response.status)


async def main():
    async with aiohttp.ClientSession() as session:
        for i in range(100):
            await post_data(session)


In [67]:
loop = asyncio.get_event_loop()
loop.create_task(main())

RuntimeError: This event loop is already running